## 필요 API


In [1]:
import requests
import json 

# GET 
def rest_get(url):
    response = requests.get(url) 
    # print(str(response.status_code) + " | " + response.text) 
    return json.loads(response.text)

def rest_get_raw(url, header):
    response = requests.get(url, headers=header) 
    return json.loads(response.text)

# POST (JSON) 
def rest_post(url, token, data):
    header = {'X-Auth-Token': token, 'Content-Type': 'application/json; chearset=utf-8'}
    response = requests.post(url, data=json.dumps(data), headers=header) 
    # print(str(response.status_code) + " | " + response.text)
    return json.loads(response.text)

def rest_post_raw(url, header, data):
    response = requests.post(url, data=json.dumps(data), headers=header) 
    return json.loads(response.text)

def rest_put_raw(url, header, data):
    response = requests.put(url, data=json.dumps(data), headers=header)
#     print(str(response.status_code) + " | " + response.text)
    return json.loads(response.text)

In [2]:
BASE_URL = 'https://huqeyhi95c.execute-api.ap-northeast-2.amazonaws.com/prod'
LEADER_BOARD_URL = 'https://si1kddo29f.execute-api.ap-northeast-2.amazonaws.com/prod/leaderboard'
X_AUTH_TOKEN = "a21ffa7f526d9fa87ec8107ddf9926bf"

## API call

In [19]:
#  start 
url = f"{BASE_URL}/start"
data = {"problem": 1}
start = rest_post(url, X_AUTH_TOKEN, data)
auth_key = start["auth_key"]

In [20]:
# waiting line API return List
def waitingLineAPI():
    url = f"{BASE_URL}/waiting_line"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    waitingLine = rest_get_raw(url, header)
    return waitingLine["waiting_line"]

In [21]:
# game result API return List
def gameResultAPI():
    url = f"{BASE_URL}/game_result"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    gameResult = rest_get_raw(url, header)
    return gameResult["game_result"]

In [22]:
# user info API return List
def userInfoAPI():
    url = f"{BASE_URL}/user_info"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    userInfo = rest_get_raw(url, header)
    return userInfo["user_info"]

In [31]:
# match API
def matchAPI(data):
    url = f"{BASE_URL}/match"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    match = rest_put_raw(url, header, data)
    return match

# data format
# data = {
#     "pairs": [[1, 2], [9, 7], [11, 49]]
# }

In [32]:
# change grade API -> decide after game result 
def changeGradeAPI(data):
    url = f"{BASE_URL}/change_grade"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    changeGrade = rest_put_raw(url, header, data)
    return changeGrade

# 등급의 범위는 0 이상 9,999 이하 정수
# data format
# data = {
#     "commands": [
#         { "id": 1, "grade": 1900 }
#          ...
#     ]
# }

In [25]:
# get score API
def scoreAPI():
    url = f"{BASE_URL}/score"
    header = {'Authorization': auth_key, 'Content-Type': 'application/json'}
    score = rest_get_raw(url, header)
    return score

In [26]:
def status():
    from pprint import pprint
    print("@ WAITING LINE")
    pprint(waitingLineAPI())
    print()
    print("@ GAME RESULT")
    pprint(gameResultAPI())

## test

In [54]:
GRADE_GAP = 2000

In [57]:
status()

@ WAITING LINE
[]

@ GAME RESULT
[]


In [58]:
userInfoAPI()

[{'id': 1, 'grade': 0},
 {'id': 2, 'grade': 0},
 {'id': 3, 'grade': 0},
 {'id': 4, 'grade': 0},
 {'id': 5, 'grade': 0},
 {'id': 6, 'grade': 0},
 {'id': 7, 'grade': 0},
 {'id': 8, 'grade': 0},
 {'id': 9, 'grade': 0},
 {'id': 10, 'grade': 0},
 {'id': 11, 'grade': 0},
 {'id': 12, 'grade': 0},
 {'id': 13, 'grade': 0},
 {'id': 14, 'grade': 0},
 {'id': 15, 'grade': 0},
 {'id': 16, 'grade': 0},
 {'id': 17, 'grade': 0},
 {'id': 18, 'grade': 0},
 {'id': 19, 'grade': 0},
 {'id': 20, 'grade': 0},
 {'id': 21, 'grade': 0},
 {'id': 22, 'grade': 0},
 {'id': 23, 'grade': 0},
 {'id': 24, 'grade': 0},
 {'id': 25, 'grade': 0},
 {'id': 26, 'grade': 0},
 {'id': 27, 'grade': 0},
 {'id': 28, 'grade': 0},
 {'id': 29, 'grade': 0},
 {'id': 30, 'grade': 0}]

In [59]:
# 매칭
from pprint import pprint
userInfo = userInfoAPI()
waitingLine = waitingLineAPI()
userDist = [{'id': dic['id'], 'from': dic['from'], 'grade': userInfo[dic['id']-1]['grade']} for dic in waitingLine]
userDist.sort(key=lambda x : (x['grade'], x['from']))
pprint(userDist)

pairs = []
idx = 0
while idx < len(userDist) - 1:
    p1 = userDist[idx]
    p2 = userDist[idx + 1]
    
    if abs(p1['grade'] - p2['grade']) < GRADE_GAP:
        pairs.append([p1['id'], p2['id']])
        idx += 1
    idx += 1

[]


In [ ]:
# 결과 확인 후 그레이드 설정


In [56]:
data = {
    "pairs": pairs
}
matchAPI(data)

{'status': 'ready', 'time': 2}

## total